# Week 07 Live Coding Demo — Statistics & Probability I 

**Contents**
1. Bernoulli & Binomial (muon detector windows) — PMF and moments
2. Poisson counting (gamma events per minute) — empirical vs theoretical PMF
3. Exponential waiting times (trigger inter-arrival) — PDF and empirical CDF
4. Gaussian noise (thermometer) — histogram, mean, and std
5. Uniform phase → cosine amplitude — transformation intuition
6. Discrete PMF from physics (Boltzmann levels) — sampling via discrete inverse CDF
7. Inverse CDF samplers: Rayleigh speeds (2D gas) and uniform points in a disk
8. Monte Carlo I: Beer–Lambert transmission vs thickness — compare to analytic
9. Monte Carlo II: Aperture acceptance (solid angle) — compare to analytic
10. Stern–Gerlach experiment (spin-1/2): beam splitting and cos²(θ/2) law


## 0) Imports, RNG seed, and plotting defaults

In [ ]:
# We centralize imports and style so the rest of the notebook is clean.
import numpy as np, matplotlib.pyplot as plt
from math import comb, factorial, sqrt, pi, exp

# Reproducible pseudo-random numbers
rng = np.random.default_rng(77)

# Minimal plot styling for consistency and readability
plt.rcParams.update({
    "figure.figsize": (6.0, 3.6),
    "axes.grid": True,
    "axes.spines.top": False,
    "axes.spines.right": False,
    "font.size": 11,
})
print("Environment ready — numpy", np.__version__)


## 1) Bernoulli → Binomial (Muon detector in short windows)

**Story**
- In a 10 ms time window, our scintillator either registers at least one muon hit (1) or no hit (0).
- That single window is a **Bernoulli** trial with success probability \(p\).
- If we collect **M** windows in a run and count successes, we get a **Binomial(M, p)** distribution.

**Goals**
- See an empirical PMF for 0/1 outcomes and compare Binomial PMF to simulation.
- Connect sample mean/variance to Binomial formulas \(\mathrm{E}[K]=Mp\), \(\mathrm{Var}[K]=Mp(1-p)\).


In [ ]:
# --- Bernoulli: estimate P(hit) from many windows ---
p_hit = 0.08    # true per-window hit probability (toy)
N = 3000        # number of windows
hits = (rng.random(N) < p_hit).astype(int)  # 0/1 outcomes

# Empirical PMF for {0,1}
vals, counts = np.unique(hits, return_counts=True)
pmf_emp = counts / counts.sum()

plt.bar(vals, pmf_emp, width=0.5, color="tab:blue")
plt.xticks([0,1], ["no hit (0)", "hit (1)"])
plt.ylim(0,1); plt.ylabel("empirical PMF")
plt.title("Muon detector: empirical PMF for a 10 ms window")
plt.show()

print(f"Estimated P(hit) ≈ {pmf_emp[vals.tolist().index(1)]:.3f}  (true p = {p_hit})")


In [ ]:
# --- Binomial: number of hits in M windows per run ---
M = 40          # windows per run
R = 25000       # number of independent runs
k_sim = rng.binomial(M, p_hit, size=R)  # simulated counts per run

# Empirical PMF (histogram-based) and analytic Binomial PMF
bins = np.arange(-0.5, M+1.5, 1)
hist, _ = np.histogram(k_sim, bins=bins, density=True)
centers = (bins[:-1] + bins[1:]) / 2
pmf_th = np.array([comb(M, k) * (p_hit**k) * ((1-p_hit)**(M-k)) for k in range(M+1)])

plt.bar(centers, hist, width=0.8, alpha=0.6, label="empirical")
plt.plot(np.arange(M+1), pmf_th, "k-", lw=2, label="Binomial theory")
plt.xlabel("hits in M=40 windows"); plt.ylabel("PMF")
plt.title("Binomial(M=40, p=0.08): simulation vs theory")
plt.legend(); plt.show()

print(f"Empirical mean = {k_sim.mean():.3f}  (theory M*p = {M*p_hit:.3f})")
print(f"Empirical var  = {k_sim.var():.3f}  (theory M*p*(1-p) = {M*p_hit*(1-p_hit):.3f})")


## 2) Poisson counting (Gamma-ray events per minute)

**Story**
- Our telescope records the number of gamma events per minute. Independent arrivals at rate \(\lambda\) suggest a Poisson model.
- For Poisson, mean equals variance \(\mathrm{E}[N]=\mathrm{Var}[N]=\lambda\).

**Goals**
- Compare empirical PMF to Poisson PMF, and check mean/variance numerically.


In [ ]:
lam = 3.5               # mean counts/min (toy but realistic order)
minutes = 25000         # number of minutes observed
counts = rng.poisson(lam, size=minutes)

k_max = int(lam + 6*sqrt(lam))  # cover most of the mass
ks = np.arange(0, k_max+1)
pmf_emp = np.array([(counts == k).mean() for k in ks])
pmf_th  = np.array([exp(-lam) * (lam**k) / factorial(k) for k in ks])

plt.bar(ks, pmf_emp, alpha=0.6, label="empirical")
plt.plot(ks, pmf_th, "k-", lw=2, label=f"Poisson(λ={lam})")
plt.xlabel("counts in 1 minute"); plt.ylabel("PMF")
plt.title("Gamma-ray count statistics")
plt.legend(); plt.show()

print(f"mean = {counts.mean():.3f}, var = {counts.var():.3f}  (Poisson: mean=var≈{lam})")


## 3) Exponential waiting times (Trigger inter-arrival)

**Story**
- In a Poisson process with rate \(\lambda\), the waiting time between events is **Exponential** with mean \(1/\lambda\).
- This “memoryless” model is common in radioactive decay and dark counts.

**Goals**
- Visualize the PDF via histogram and compare empirical CDF to the theoretical CDF.


In [ ]:
lam = 0.8                # events per second
N = 15000
wait = rng.exponential(scale=1/lam, size=N)

# PDF view
plt.hist(wait, bins=60, density=True, alpha=0.6, label="samples")
x = np.linspace(0, np.quantile(wait, 0.99), 300)
plt.plot(x, lam*np.exp(-lam*x), "r--", lw=2, label="λ e^{-λx}")
plt.xlabel("waiting time [s]"); plt.ylabel("PDF")
plt.title("Exponential waiting times (λ=0.8 s⁻¹)")
plt.legend(); plt.show()

# Empirical CDF vs theory
xs = np.sort(wait)
ecdf = np.arange(1, N+1)/N
F_theory = 1 - np.exp(-lam*xs)
plt.plot(xs, ecdf, label="empirical CDF")
plt.plot(xs, F_theory, "r--", label="theory CDF")
plt.xlabel("waiting time [s]"); plt.ylabel("CDF")
plt.title("Exponential empirical CDF vs theory")
plt.legend(); plt.show()


## 4) Gaussian noise (Thermometer around a stable bath)

**Story**
- A bath is stabilized at 20.00 °C. Our thermometer has Gaussian readout noise with σ≈0.05 °C.

**Goals**
- See a Gaussian-shaped histogram; compute the sample mean and standard deviation to summarize noise.


In [ ]:
mu_true, sigma = 20.00, 0.05
N = 5000
T = mu_true + rng.normal(0, sigma, size=N)

plt.hist(T, bins=40, density=True, alpha=0.6, label="samples")
tgrid = np.linspace(mu_true-0.3, mu_true+0.3, 300)
pdf = (1/(sigma*sqrt(2*pi)))*np.exp(-(tgrid-mu_true)**2/(2*sigma**2))
plt.plot(tgrid, pdf, "k--", lw=2, label="N(μ,σ²) theory")
plt.xlabel("temperature [°C]"); plt.ylabel("PDF")
plt.title("Thermometer noise about a stable bath")
plt.legend(); plt.show()

print("sample mean =", round(T.mean(), 4), "  sample std =", round(T.std(ddof=1), 4))


## 5) Uniform phase → cosine amplitude (Transformation intuition)

**Story**
- In wave interference, phase \(\phi\sim\mathrm{Uniform}(0,2\pi)\).
- Amplitude \(A=\cos\phi\) is **not** uniform; it clusters near ±1 because many phases map near those values.

**Goals**
- Compare the flat phase histogram to the non-uniform amplitude histogram.


In [ ]:
phi = rng.uniform(0, 2*np.pi, 25000)  # uniform phase
A = np.cos(phi)                        # induced amplitude

fig, ax = plt.subplots(1,2, figsize=(9,3.6))
ax[0].hist(phi, bins=40, density=True, alpha=0.7, color="tab:blue")
ax[0].set_title("Uniform phase φ"); ax[0].set_xlabel("φ [rad]"); ax[0].set_ylabel("PDF")

ax[1].hist(A, bins=40, density=True, alpha=0.7, color="tab:orange")
ax[1].set_title("Amplitude A = cos(φ)"); ax[1].set_xlabel("A"); ax[1].set_ylabel("PDF")
plt.tight_layout(); plt.show()


## 6) Discrete PMF from physics (Boltzmann levels)

**Story**
- Energy levels \(E_n=n\Delta E\) in thermal equilibrium at \(kT\) have weights \(w_n\propto e^{-E_n/kT}\).

**Goals**
- Build a discrete PMF, sample from it using a **discrete inverse CDF**, and compare empirical frequencies to the PMF.


In [ ]:
kT = 0.25               # thermal energy unit (ΔE absorbed into units)
n_max = 12
E = np.arange(0, n_max+1)
w = np.exp(-E/kT)
pmf = w / w.sum()

# Discrete inverse CDF sampling
N = 60000
u = rng.random(N)
cdf = np.cumsum(pmf)

# The inverse-CDF step:
# For each u[i], return the smallest index n with cdf[n] >= u[i].
# This maps the unit interval into integer bins whose lengths equal pmf[n].
# np.searchsorted does exactly that "leftmost insertion" search in O(log n) per u.
states = np.searchsorted(cdf, u) # vectorized: returns an array of indices in 0..n_max

vals, counts = np.unique(states, return_counts=True)
freq = counts / counts.sum()

plt.stem(np.arange(n_max+1), pmf, basefmt=" ", linefmt="k-", markerfmt="ko", label="PMF (theory)")
plt.bar(vals, freq, alpha=0.5, label="empirical", width=0.6)
plt.xlabel("level index n"); plt.ylabel("probability")
plt.title("Boltzmann occupancy over discrete levels")
plt.legend(); plt.show()

print("Empirical mean level ≈", round(states.mean(), 3))


## 7) Inverse CDF samplers (Rayleigh speeds & uniform points in a disk)

**Story A — Rayleigh speeds (2D gas):**
- If velocity components are independent \(N(0,\sigma^2)\), the speed \(v\) has **Rayleigh** PDF \(f(v)=\frac{v}{\sigma^2}e^{-v^2/(2\sigma^2)}\).

**Story B — Uniform points in a circular detector:**
- To be uniform in area, radius \(r\) must follow \(f_r(r)=2r/R^2\), i.e., use \(r=R\sqrt{U}\).

**Goals**
- Implement inverse-CDF samplers and verify with histograms/overlays.


In [ ]:
# (A) Rayleigh via inverse CDF
sigma = 1.2
U = rng.random(40000)
V = sigma * np.sqrt(-2*np.log(1-U)) # F^-1(U)

# (B) Uniform-in-disk via r=R*sqrt(U), theta~Uniform
R = 1.0
U2 = rng.random(25000); theta = rng.uniform(0, 2*np.pi, 25000)
r = R * np.sqrt(U2) # F^-1(U)
x, y = r*np.cos(theta), r*np.sin(theta)

fig, ax = plt.subplots(1,2, figsize=(9,3.6))
# Rayleigh
ax[0].hist(V, bins=60, density=True, alpha=0.6, label="samples")
v = np.linspace(0, V.max(), 300)
pdf_ray = (v/(sigma**2)) * np.exp(-v**2/(2*sigma**2))
ax[0].plot(v, pdf_ray, "k--", lw=2, label="Rayleigh PDF")
ax[0].set_xlabel("speed v"); ax[0].set_ylabel("PDF"); ax[0].set_title("2D gas speeds (Rayleigh)"); ax[0].legend()

# Disk points + outline
ax[1].scatter(x, y, s=3, alpha=0.35)
ax[1].add_artist(plt.Circle((0,0), R, fill=False, color="k"))
ax[1].set_aspect("equal", adjustable="box"); ax[1].set_title("Uniform points in a detector disk")

plt.tight_layout(); plt.show()

# Radial distribution check
fig, ax = plt.subplots()
ax.hist(r, bins=40, range=(0,R), density=True, alpha=0.6, label="empirical f_r(r)")
rr = np.linspace(0, R, 200); fr = 2*rr/(R**2)
ax.plot(rr, fr, "k--", lw=2, label="theory 2r/R^2")
ax.set_xlabel("radius r"); ax.set_ylabel("PDF"); ax.set_title("Radial PDF inside a disk")
ax.legend(); plt.show()


## 8) Monte Carlo I — Beer–Lambert transmission through an absorbing slab

**Story**
- Photons traverse a slab of thickness $L$ with attenuation coefficient $\mu$.
- Free path $S\sim\mathrm{Exp}(\mu)$. Transmission event is $\{S > L\}$.

**Goal**
- Compare Monte Carlo transmission to the analytic $e^{-\mu L}$ curve.



In [ ]:
mu = 0.7                                   # attenuation coefficient [1/cm]
L_vals = np.linspace(0, 5, 25)             # list of slab thicknesses to evaluate
N = 60_000                                  # photons per thickness = # of trials for each L
trans_mc = []                               # store Monte Carlo transmission at each L

for L in L_vals:
    S = rng.exponential(scale=1/mu, size=N) # draw N free paths S ~ Exp(mu); mean free path = 1/mu
    trans_mc.append((S > L).mean())         # estimator T̂(L) = fraction of trials with S > L (survive slab)

trans_mc = np.array(trans_mc)               # shape matches L_vals
trans_th = np.exp(-mu * L_vals)             # analytic Beer–Lambert transmission T(L) = e^{-mu L}

plt.plot(L_vals, trans_th, "k--", lw=2, label="analytic e^{-μL}")
plt.plot(L_vals, trans_mc, "o-", label="MC estimate")
plt.xlabel("thickness L [cm]"); plt.ylabel("transmission fraction")
plt.title("Beer–Lambert: transmission vs thickness")
plt.legend(); plt.show()

print("Mean absolute error (MC vs theory):", float(np.mean(np.abs(trans_mc - trans_th))))


## 9) Monte Carlo II — Acceptance of a circular aperture (solid angle fraction)

**Story**
- A point source at the origin emits isotropically.
- A circular aperture of radius \(a\) is centered at \(z=d\). A ray hits if the intersection lies within the disk.

**Goal**
- Estimate acceptance via Monte Carlo and compare to analytic solid angle $\displaystyle \Omega = 2\pi\!\left(1-\frac{d}{\sqrt{d^{2}+a^{2}}}\right)$  
(fraction $= \Omega/(4\pi)$).


In [ ]:
def acceptance_mc(a_over_d=0.5, N=1_000_000):
    d = 1.0
    a = a_over_d * d
    u = rng.uniform(-1, 1, N)                # u = cosθ over full sphere
    u_thr = d / np.sqrt(d**2 + a**2)         # cosθ_max
    hit = u >= u_thr                          # already excludes back hemisphere
    return hit.mean()                         # fraction over ALL directions


ratios = np.linspace(0.1, 1.5, 15)
acc = np.array([acceptance_mc(r) for r in ratios])

d = 1.0
a = ratios * d
Omega = 2*np.pi*(1 - d/np.sqrt(d**2 + a**2))
frac = Omega/(4*np.pi)

plt.plot(ratios, frac, "k--", lw=2, label="analytic Ω/(4π)")
plt.plot(ratios, acc, "o-", label="MC acceptance")
plt.xlabel("aperture radius / distance (a/d)"); plt.ylabel("acceptance fraction")
plt.title("Isotropic point source → circular aperture acceptance")
plt.legend(); plt.show()

print("Max abs diff (MC vs analytic):", float(np.max(np.abs(acc-frac))))


## 10) Stern–Gerlach (spin‑1/2) — beam splitting and cos²(θ/2) law

**Story**
- A neutral silver atom beam passes through a magnetic field gradient; spins split into **spin‑up** and **spin‑down** along the gradient axis (call it **z**).
- If we send the **up** beam into a second analyzer rotated by angle **θ** relative to z, the probability of measuring **up** is $\cos^2(\tfrac{\theta}{2})$.

**Goals**
1. Visualize spatial beam splitting into two Gaussian spots (up/down).
2. Sweep θ and verify the **cos²(θ/2)** detection probability empirically.


In [ ]:
# --- 10a) Spatial splitting into two Gaussian lobes ---
# We model the transverse deflection as ±Δ along y for spin up/down, with Gaussian beam width σ_beam.
N = 6000
sigma_beam = 0.25
delta = 1.5  # separation between up/down centroids
# half the atoms are up, half down (unpolarized source)
labels = rng.integers(0, 2, size=N)  # 0=down, 1=up
# x is beam axis coordinate on detector plane; y is deflection
x = rng.normal(0, 0.2, size=N)
y = rng.normal(0, sigma_beam, size=N) + np.where(labels==1, +delta/2, -delta/2)

plt.scatter(x[labels==1], y[labels==1], s=6, alpha=0.4, label="spin up")
plt.scatter(x[labels==0], y[labels==0], s=6, alpha=0.4, label="spin down")
plt.axhline(0, color="k", lw=1, alpha=0.4)
plt.xlabel("x (beam axis coordinate)"); plt.ylabel("y (deflection)")
plt.title("Stern–Gerlach: two spots from spin‑1/2 beam")
plt.legend(); plt.gca().set_aspect("equal", adjustable="box"); plt.show()


In [ ]:
# --- 10b) Second analyzer at angle θ: P(up|θ) = cos^2(θ/2) ---
# Start with a *pure* up_z beam (e.g., by blocking the down spot). Then analyze along axis rotated by θ.
def simulate_up_probability(theta_deg, N=20000):
    theta = np.deg2rad(theta_deg)
    p_up = np.cos(theta/2)**2          # Born rule for spin-1/2
    # Simulate N Bernoulli trials with success prob p_up
    return rng.binomial(N, p_up)/N

thetas = np.linspace(0, 180, 19)
p_emp = np.array([simulate_up_probability(t) for t in thetas])
p_the = np.cos(np.deg2rad(thetas)/2)**2

plt.plot(thetas, p_the, "k--", lw=2, label="theory cos²(θ/2)")
plt.plot(thetas, p_emp, "o-", label="empirical")
plt.xlabel("rotation angle θ [deg]"); plt.ylabel("P(up along θ)")
plt.title("Stern–Gerlach: second analyzer probability")
plt.legend(); plt.show()

print("Mean abs deviation (empirical vs theory):", float(np.mean(np.abs(p_emp - p_the))))
